## sliding-puzzle

O *sliding-puzzle* é um quebra-cabeça deslizante que consiste em uma grade $N\times N$ na qual $N^2-1$ peças numeradas são colocadas aleatoriamente em $N^2-1$ das $N^2$ posições e uma posição está vazia, permitindo que as peças adjacentes sejam movidas para ocupar o espaço vazio, com o objetivo final de reorganizar as peças em uma ordem numérica específica, tipicamente de 1 a $N-1$, seguindo um número mínimo de movimentos.

Observe que em $N\times N$ posições podemos permutar as $N^2$ peças (contando o espaço vazio como peça), isto significa que temos $N!$ combinações possíveis. No entanto, nem todas essas permutações são possíveis no sliding-puzzle, pois algumas configurações não são alcançáveis a partir de outras por meio dos movimentos válidos.

Utilizaremos este jogo para exemplificar algumas das técnicas de busca cega e heurística.

## Definição da classe sliding-puzzle

A classe definida a seguir é uma estrutura de dados que encapsula as funcionalidades necessárias para representar e manipular um quebra-cabeça sliding-puzzle.

In [1]:
class SlidingPuzzle:
    def __init__(self, initial_state):
        self.state = initial_state

    def __str__(self):
        str_rep = ""
        for row in self.state:
            str_rep += " ".join(map(str, row)) + "\n"
        return str_rep
    
    def move(self, direction):
        # Encontra a posição do espaço vazio
        empty_pos = self.find_empty()

        # Verifica a direção e troca a posição do espaço vazio com a peça adjacente na direção especificada
        if direction == 'up' and empty_pos[0] > 0:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]-1][empty_pos[1]] = \
                self.state[empty_pos[0]-1][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'down' and empty_pos[0] < 2:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]+1][empty_pos[1]] = \
                self.state[empty_pos[0]+1][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'left' and empty_pos[1] > 0:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]-1] = \
                self.state[empty_pos[0]][empty_pos[1]-1], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'right' and empty_pos[1] < 2:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]+1] = \
                self.state[empty_pos[0]][empty_pos[1]+1], self.state[empty_pos[0]][empty_pos[1]]

    def get_state(self):
        return tuple(map(tuple, self.state))

    def find_empty(self):
        for i in range(len(self.state)):
            for j in range(len(self.state[0])):
                if self.state[i][j] == 0:
                    return (i, j)
                
    # Busca possíveis movimentos com possibilidade de descartar o ultimo movimento realizado
    def get_possible_moves(self, last_move='none'):
        possible_moves = []
        empty_pos = self.find_empty()
        if empty_pos[0] > 0 and last_move!='down':
            possible_moves.append('up')
        if empty_pos[0] < len(self.state)-1 and last_move!='up':
            possible_moves.append('down')
        if empty_pos[1] > 0 and last_move!='right':
            possible_moves.append('left')
        if empty_pos[1] < len(self.state)-1 and last_move!='left':
            possible_moves.append('right')
        return possible_moves
    
    def get_inverse(self, move):
        if move == 'up':
            return 'down'
        if move == 'down':
            return 'up'
        if move == 'left':
            return 'right'
        if move == 'right':
            return 'left'
        return None

Exemplo de uso da classe Puzzle8:

In [2]:
# Exemplo de uso
initial_state = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

puzzle = SlidingPuzzle(initial_state)
print("Estado inicial:")
print(puzzle)

# Movendo a peça vazia para cima
puzzle.move('up')
print("Depois de mover para cima:")
print(puzzle)
puzzle.move('down')
print("Depois de mover para baixo:")
print(puzzle)
puzzle.move('right')
print("Depois de mover para direita:")
print(puzzle)
puzzle.move('left')
print("Depois de mover para esquerda:")
print(puzzle)

Estado inicial:
1 2 3
4 0 5
6 7 8

Depois de mover para cima:
1 0 3
4 2 5
6 7 8

Depois de mover para baixo:
1 2 3
4 0 5
6 7 8

Depois de mover para direita:
1 2 3
4 5 0
6 7 8

Depois de mover para esquerda:
1 2 3
4 0 5
6 7 8



## Busca Cega

### Backtracking

In [3]:
class BacktrackingSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        return self.backtrack()

    # Função de busca por força bruta
    def backtrack(self, last_move='none'):
        state = self.puzzle.get_state()
        if self.is_goal(state):
            return [state]
        for move in self.puzzle.get_possible_moves(last_move):
            self.puzzle.move(move)
            last_move = move
            solution = self.backtrack(last_move)
            if solution:
                return [state] + solution
            else:
                reverse = self.puzzle.get_inverse(move)
                self.puzzle.move(reverse)
                    
        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [4]:
# Exemplo de uso
#initial_state = [[2,8,3], [1,6,4], [7,0,5]]
#goal_state = [[1,2,3],[8,0,4],[7,6,5]]
initial_state = [[3,2], [0,1]]
goal_state = [[0,1],[2,3]]
puzzle = SlidingPuzzle(initial_state)
solver = BacktrackingSolver(puzzle, goal_state)
solution = solver.solve()

if solution:
    print("Solução encontrada:")
    for step, state in enumerate(solution):
        print(f"Passo {step+1}: {state}")
else:
    print("Não foi possível encontrar uma solução.")


Solução encontrada:
Passo 1: ((3, 2), (0, 1))
Passo 2: ((0, 2), (3, 1))
Passo 3: ((2, 0), (3, 1))
Passo 4: ((2, 1), (3, 0))
Passo 5: ((2, 1), (0, 3))
Passo 6: ((0, 1), (2, 3))


In [5]:
from itertools import permutations

# Definindo os valores possíveis
valores = [0, 1, 2, 3]

# Gerando todas as permutações possíveis
perms = list(permutations(valores))

# Mostrando as permutações
for p in perms:
    print(p)


(0, 1, 2, 3)
(0, 1, 3, 2)
(0, 2, 1, 3)
(0, 2, 3, 1)
(0, 3, 1, 2)
(0, 3, 2, 1)
(1, 0, 2, 3)
(1, 0, 3, 2)
(1, 2, 0, 3)
(1, 2, 3, 0)
(1, 3, 0, 2)
(1, 3, 2, 0)
(2, 0, 1, 3)
(2, 0, 3, 1)
(2, 1, 0, 3)
(2, 1, 3, 0)
(2, 3, 0, 1)
(2, 3, 1, 0)
(3, 0, 1, 2)
(3, 0, 2, 1)
(3, 1, 0, 2)
(3, 1, 2, 0)
(3, 2, 0, 1)
(3, 2, 1, 0)


In [6]:
from itertools import permutations

# Definindo os valores possíveis
valores = [0, 1, 2, 3]

# Gerando todas as permutações possíveis
perms = list(permutations(valores))

# Filtrando as permutações pares
perms_pares = [p for i, p in enumerate(perms) if i % 2 == 0]

# Mostrando as permutações pares
for p in perms_pares:
    print(p)


(0, 1, 2, 3)
(0, 2, 1, 3)
(0, 3, 1, 2)
(1, 0, 2, 3)
(1, 2, 0, 3)
(1, 3, 0, 2)
(2, 0, 1, 3)
(2, 1, 0, 3)
(2, 3, 0, 1)
(3, 0, 1, 2)
(3, 1, 0, 2)
(3, 2, 0, 1)
